In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

In [2]:
train_data = pd.read_csv('../input/train.csv')
train_data.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074


In [3]:
one_hot = pd.get_dummies(train_data['type'])
train_data.drop(['type'],axis = 1,inplace=True)
train_data = train_data.join(one_hot)
train_data.head()

,id,molecule_name,atom_index_0,atom_index_1,scalar_coupling_constant,1JHC,1JHN,2JHC,2JHH,2JHN,3JHC,3JHH,3JHN
0,0,dsgdb9nsd_000001,1,0,84.8076,1,0,0,0,0,0,0,0
1,1,dsgdb9nsd_000001,1,2,-11.2570,0,0,0,1,0,0,0,0
2,2,dsgdb9nsd_000001,1,3,-11.2548,0,0,0,1,0,0,0,0
3,3,dsgdb9nsd_000001,1,4,-11.2543,0,0,0,1,0,0,0,0
4,4,dsgdb9nsd_000001,2,0,84.8074,1,0,0,0,0,0,0,0


In [4]:
target = train_data['scalar_coupling_constant'].values
train_data.drop(['id','molecule_name','scalar_coupling_constant', '3JHN'], axis=1, inplace=True)
features = train_data.iloc[:,:].values

model = RandomForestRegressor(random_state=42, n_jobs=-1, verbose=1)
model.fit(features, target)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   44.9s finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=42, verbose=1, warm_start=False)

In [5]:
test_data=pd.read_csv('../input/test.csv')
test_data.head()

,id,molecule_name,atom_index_0,atom_index_1,type
0,4658147,dsgdb9nsd_000004,2,0,2JHC
1,4658148,dsgdb9nsd_000004,2,1,1JHC
2,4658149,dsgdb9nsd_000004,2,3,3JHH
3,4658150,dsgdb9nsd_000004,3,0,1JHC
4,4658151,dsgdb9nsd_000004,3,1,2JHC


In [6]:
test_data.drop(['id','molecule_name'], axis=1, inplace=True)
test_data.head()

,atom_index_0,atom_index_1,type
0,2,0,2JHC
1,2,1,1JHC
2,2,3,3JHH
3,3,0,1JHC
4,3,1,2JHC


In [7]:
one_hot = pd.get_dummies(test_data['type'])
test_data.drop(['type'],axis = 1,inplace=True)
test_data = test_data.join(one_hot)
test_data.head()

,atom_index_0,atom_index_1,1JHC,1JHN,2JHC,2JHH,2JHN,3JHC,3JHH,3JHN
0,2,0,0,0,1,0,0,0,0,0
1,2,1,1,0,0,0,0,0,0,0
2,2,3,0,0,0,0,0,0,1,0
3,3,0,1,0,0,0,0,0,0,0
4,3,1,0,0,1,0,0,0,0,0


In [8]:
test_data.drop(['3JHN'], axis=1, inplace=True)

In [9]:
predictions = model.predict(test_data.iloc[:,:].values)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.9s finished


In [10]:
submission = pd.read_csv('../input/sample_submission.csv')
submission.head()

,id,scalar_coupling_constant
0,4658147,0
1,4658148,0
2,4658149,0
3,4658150,0
4,4658151,0


In [11]:
submission['scalar_coupling_constant'] = predictions
submission.to_csv('basline_catboost_submission.csv', index=False)